## Multi-Qubit Quantum Gates / Operators

Now that we know how to work with single-qubit gates we move on to multi-qubit operators which act on more than one qubit/statevector. 

Remember a multi-qubit state $$|XY\rangle$$ is the tensor product

$$ |XY\rangle = |X\rangle \otimes |Y\rangle = |X\rangle|Y\rangle$$

---

### SWAP 
$$ \text{SWAP} |a\rangle |b\rangle = |b\rangle |a\rangle $$

It's matrix representation 

$$ \text{SWAP} \equiv \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 0 & 1 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \end{pmatrix} $$

### Controlled Operations

A controlled $U$ operator performs an action on a **target** qubit depending on the state of a **control** qubit. 

When dealing with the computational basis of $|0\rangle$ and $|1\rangle$ that defines each statevector, a controlled $U$ operation takes the form 

$$ CU = |0\rangle \langle 0| \otimes I_R + |1\rangle\langle 1| \otimes U $$

To make this idea clearer, let's discuss the CNOT gate. 

#### CX / CNOT

The action of a CNOT gate flips the target qubit if the control qubit is in the state $|1\rangle$ and leaves the target qubit alone if the control qubit is $|0\rangle$. To put it cleanly, 

| Before | After |
| :------: | :-----: |
| Control /  Target | Control / Target |
| $|0\rangle$   /   $|0\rangle$ | $|0\rangle$ / $|0\rangle$ |
| $|0\rangle$   /   $|1\rangle$ | $|0\rangle$ / $|1\rangle$ |
| $|1\rangle$   /   $|0\rangle$ | $|1\rangle$ / $|1\rangle$ |
| $|1\rangle$   /   $|1\rangle$ | $|1\rangle$ / $|0\rangle$ |

It leaves the control qubit alone regardless. In matrix representation 

$$ CNOT = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{pmatrix} $$

We can perform Controlled Operations using any single or even multi-qubit gate, and also make the control qubit a control multi-qubit system instead. 

We can also make the control qubit a two-qubit system but the target qubit only a single qubit. For example the CCNOT (controlled-controlled NOT) gate, also known as the Toffoli Gate -- a CNOT gate with *2* control qubits and *1* target qubit. That is, the target qubit gets inverted if and only if *both* control qubits are in the state $|1\rangle$. 

---

#### Tensor Products

Let's create the multi-qubit statevector $|01\rangle$ as a tensor product from $|0\rangle \otimes |1\rangle$. 

In [4]:
from qiskit.quantum_info import Statevector
from numpy import sqrt

zero, one = Statevector.from_label("0"), Statevector.from_label("1")
zero.tensor(one).draw("latex")

<IPython.core.display.Latex object>

Let's create the vectors representing the states $|+\rangle$ and $\frac{1}{\sqrt{2}}(|0\rangle + i|1\rangle)$ states and combine them to form a new state vector $|\psi\rangle$. 

In [5]:
plus = Statevector.from_label("+")
i_state = Statevector([1/sqrt(2), 1j/sqrt(2)])

psi = plus.tensor(i_state)
psi.draw("latex")

<IPython.core.display.Latex object>

We can also take the tensor product of operators. The tensor product $C$ of the operators $C = X \otimes Y$ is an operator that can act on simultaneously on the 2 qubit system that is equivalent  to applying $X$ on the first qubit and $Y$ on the second and taking the tensor product after. 

To put it cleanly, the following circuit: 

In [7]:
from qiskit import QuantumCircuit

circuit = QuantumCircuit(2)

circuit.x(0)
circuit.y(1) 
display(circuit.draw())

┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ Y ├
     └───┘

is the same as applying the $C = X \otimes Y$ gate on the tensor product between both `q_0` and `q_1`. 

We can generate tensor products between operators in a similar fashion as for statevectors.

In [10]:
from qiskit.quantum_info import Operator

X = Operator([[0, 1], [1, 0]])
Y = Operator([[0, -1j], [1j, 0]])

C = X.tensor(Y)
print(C)

Operator([[0.+0.j, 0.-0.j, 0.+0.j, 0.-1.j],
          [0.+0.j, 0.+0.j, 0.+1.j, 0.+0.j],
          [0.+0.j, 0.-1.j, 0.+0.j, 0.-0.j],
          [0.+1.j, 0.+0.j, 0.+0.j, 0.+0.j]],
         input_dims=(2, 2), output_dims=(2, 2))


Let's show that applying $X$ on qubit $|0\rangle$ and $Y$ on qubit $|1\rangle$ is the same as applying $C = X \otimes Y$ to $|01\rangle = |0\rangle \otimes |1\rangle$. 

In [11]:
ket0 = Statevector([1, 0])
ket1 = Statevector([0, 1])
ket_combined = ket0.tensor(ket1)

# individually applying X and Y to |0> and |1> separately
ket0f = ket0.evolve(X)
ket1f = ket1.evolve(Y)
ket_combinedf = ket0f.tensor(ket1f) 

print("X|0> ⊗ Y|1> ")
display(ket_combinedf.draw("latex"))

# applying combined operator to both, 
# redefining ket_combinedf using ket_combined 
ket_combinedf = ket_combined.evolve(X ^ Y) # X ^ Y = X ⊗ Y = C

print("(X ⊗ Y)|01>")
display(ket_combinedf.draw("latex"))

X|0> ⊗ Y|1> 


<IPython.core.display.Latex object>

(X ⊗ Y)|01>


<IPython.core.display.Latex object>

They're the same! Let's go back to the $|\psi\rangle$ vector we defined earlier, 

In [12]:
psi.draw("latex")

<IPython.core.display.Latex object>

Let's create a CNOT operator and calculate CNOT$|\psi\rangle$, or $CX|\psi\rangle$

In [13]:
CX = Operator(
    [
        [1, 0, 0, 0], 
        [0, 1, 0, 0], 
        [0, 0, 0, 1],
        [0, 0, 1, 0], 
    ]
)

psi.evolve(CX).draw("latex")

<IPython.core.display.Latex object>

Compare the coefficients of the $|10\rangle$ and $|11\rangle$ states.

---

### Partial Measurements

In the previous chapter, we used the `measure` method to simulate a measurement of a quantum statevector. This method returned the measured eigenvalue, and the resultant collapsed statevector post-measurement. 

By default, `measure` measures all qubits in the statevector, but we can provide a list of integers to *only* measure the qubits *at* those indices. To demonstrate, the cell below creates the state 

$$ W = \frac{1}{\sqrt{3}} ( |001\rangle + |010\rangle + |100\rangle). $$

In [15]:
W = Statevector([0, 1, 1, 0, 1, 0, 0, 0] / sqrt(3))
W.draw("latex")

<IPython.core.display.Latex object>

If you are confused how the above state was generated using `Statevector`, each indicy of 0 or 1 indicates which of the possible 3-qubit states we define: 

- $|000\rangle$ (index 0)
- $|001\rangle$ (index 1)
- $|010\rangle$ (index 2)
- $|011\rangle$ (index 3)
- $|100\rangle$ (index 4)
- $\vdots$ $\vdots$

As you can see, where each of these indices have a 1, defines which 3-qubits state we have. The above statevector $W$ has a 1 in index 1, 2, and 4. 

Let's simulate a measurement on the **rightmost** qubit (which has index 0) -- the opposite of normal convention (I don't know why, but there's probably a reason). 

In [17]:
eigenvalue, new_statevector = W.measure([0]) # measure qubit 0

print(f"Measured: {eigenvalue}\nState after measurement:")
new_statevector.draw("latex")

Measured: 0
State after measurement:


<IPython.core.display.Latex object>

Run the above cell a few times to see different results. Notice that measuring a `1` means we know both the other qubits are $|0\rangle$, but measuring a `0` means the remaining two qubits are in the state $$ \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle). $$

---